In [1]:
!pip install tensorboard
!pip install torch 

     |████████████████████████████████| 735.5 MB 2.1 kB/s eta 0:00:012     |██████████                      | 229.4 MB 62.9 MB/s eta 0:00:09     |███████████▉                    | 270.9 MB 62.9 MB/s eta 0:00:08     |████████████                    | 274.7 MB 62.9 MB/s eta 0:00:08��█▏            | 441.0 MB 78.3 MB/s eta 0:00:04     |███████████████████████▎        | 533.8 MB 75.0 MB/s eta 0:00:03


In [2]:
import torch 

In [4]:
!tensorboard --logdir=checkpoint/


# Webpage Address: https://<sagemaker-notebook-name>.notebook.us-east-1.sagemaker.aws/proxy/6006/#scalars&_smoothingWeight=0


TensorBoard 1.15.0 at http://ip-172-16-122-161:6006/ (Press CTRL+C to quit)
W0412 14:22:59.565724 140186897827584 checkpoint_management.py:288] FailedPreconditionError: checkpoint; Is a directory
W0412 14:22:59.565951 140186897827584 checkpoint_management.py:289] checkpoint/../checkpoint: Checkpoint ignored
^C


In [50]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import json

def extract_metrics(filename):

    model_compare = []

    with open(filename,'r') as f:
        lines = f.readlines()
        #print(lines)

        json_str = str(lines[0]).replace("'", '"')
        
    return json.loads(json_str)

def pull_final_loss(folder):
    
    sub_folds = os.listdir(folder)
    sub_folds.remove('.ipynb_checkpoints')
    
    runs = []
    params = []
    steps = []
    loss = []
    
    for f in sub_folds:
        file = folder + f
        
        event_acc = EventAccumulator(file)
        event_acc.Reload()


        # E. g. get wall clock, number of steps and value for a scalar 'Accuracy'
        w_times, step_nums, vals = zip(*event_acc.Scalars('loss'))
        
        runs.append(f)
        steps.append(step_nums[len(step_nums)-1])
        loss.append(vals[len(vals)-1])
        
        # get hyper param info
        params.append(extract_metrics('results/Hypertuning/SQUAD/'+f+'.txt'))

    loss_df = pd.DataFrame(list(zip(runs, params, steps, loss)), columns = ['Run', 'Parameters', 'Steps', 'Loss'])
    
    return loss_df


losses = pull_final_loss('checkpoint/')
losses.to_csv('GPT2_SQUAD.csv')